In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service

# **1️⃣ Set Up Web Driver**
chrome_driver_path = r"C:\chromedriver-win64\chromedriver.exe"  # Update this
service = Service(chrome_driver_path)
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Runs in background for speed
driver = webdriver.Chrome(service=service, options=options)

# **2️⃣ Open the Stats Page**
url = "https://fbref.com/en/comps/Big5/stats/players/Big-5-European-Leagues-Stats"
driver.get(url)

# **3️⃣ Wait for Table to Load**
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "stats_standard")))
print("✅ Page loaded successfully.")

# **4️⃣ Scroll to Load All Players**
print("📜 Scrolling to load all players...")
for _ in range(10):  # Scroll 10 times to load all rows
    driver.execute_script("window.scrollBy(0, 5000);")
    time.sleep(2)  # Allow time for content to load

print("✅ Scrolling complete.")

# **5️⃣ Locate the Table & Extract Data**
table_xpath = '//*[@id="stats_standard"]/tbody'
table = driver.find_element(By.XPATH, table_xpath)
rows = table.find_elements(By.TAG_NAME, "tr")  # Get all rows at once

print(f"📊 Found {len(rows)} rows. Extracting data...")

# **6️⃣ Extract Data Efficiently**
data = []
for row in rows:
    cols = row.find_elements(By.TAG_NAME, "td")  # Get all columns
    if cols:
        data.append([col.text for col in cols])

# **7️⃣ Convert to DataFrame & Save**
columns = [
    "Player", "Nation", "Position", "Squad", "Comp", "Age", "Birth Year", "Games",
    "Starts", "Minutes", "Gls", "Ast", "G+A", "NP Gls", "PK", "PKatt", "Yellow",
    "Red", "xG", "npxG", "xAG", "npxG+xAG", "PrgC", "PrgP", "PrgR"
]

df = pd.DataFrame(data, columns=columns)
df.to_csv("all_fb_players.csv", index=False, encoding="utf-8")
print(f"✅ Data saved to 'all_fb_players.csv' ({len(df)} rows).")

# **8️⃣ Close Browser**
driver.quit()

In [12]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

# **1️⃣ Set Up Web Driver**
chrome_driver_path = r"C:\chromedriver-win64\chromedriver.exe"  # Update path
service = Service(chrome_driver_path)
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Runs without opening browser (much faster)
driver = webdriver.Chrome(service=service, options=options)

# **2️⃣ Open the FBRef Stats Page**
url = "https://fbref.com/en/comps/Big5/stats/players/Big-5-European-Leagues-Stats"
driver.get(url)
time.sleep(5)  # Allow time for page to load

# **3️⃣ JavaScript to Extract Table Data in One Go**
table_script = """
let table = document.querySelector("#stats_standard tbody");
let rows = table.querySelectorAll("tr");
let data = [];
rows.forEach(row => {
    let cols = row.querySelectorAll("th, td");
    let rowData = [];
    cols.forEach(col => rowData.push(col.innerText.trim()));
    data.push(rowData);
});
return data;
"""

# **4️⃣ Execute JavaScript to Get Data**
players_data = driver.execute_script(table_script)

# **5️⃣ Convert to DataFrame**
columns = [
    "Rk", "Player", "Nation", "Position", "Squad", "Comp", "Age", "Birth Year", "Games",
    "Starts", "Minutes", "Nintys", "Gls", "Ast", "G+A","G-PK", "PK", "PKatt", "Yellow", "Red", "xG", "npxG", "xAG", "npxG+xAG", "PrgC", "PrgP", "PrgR","Glss","Asts","G+As","G-PKs","G+A-PK","xGs","xAGs","xG+xAG","npxGs","npxG+xAG","Matches"
]
df = pd.DataFrame(players_data, columns=columns)

# **6️⃣ Save to CSV**
df.to_csv("all_fb_players_fast.csv", index=False, encoding="utf-8")
print(f"✅ Data saved to 'all_fb_players_fast.csv' ({len(df)} rows).")

# **7️⃣ Close Browser**
driver.quit()

✅ Data saved to 'all_fb_players_fast.csv' (2827 rows).


In [17]:
import re
import requests

from bs4 import BeautifulSoup

In [19]:
import requests
from bs4 import BeautifulSoup

# URL of the page to scrape
url = "https://www.transfermarkt.com/transfers/transferrekorde/statistik?saison_id=alle&land_id=0&ausrichtung=&spielerposition_id=&altersklasse=&leihe=&w_s=&plus=1"

# Headers to mimic a browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Send a GET request to the URL
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the table containing the player data
    table = soup.find('table', {'class': 'items'})

    # Initialize lists to store the data
    players = []
    fees = []

    # Loop through each row in the table
    for row in table.find_all('tr')[1:]:  # Skip the header row
        columns = row.find_all('td')

        # Debug: Print the number of columns in the row
        print(f"Number of columns: {len(columns)}")

        # Ensure the row has enough columns
        if len(columns) >= 7:  # Check if there are at least 7 columns
            # Extract player name
            player_name = columns[1].find('a').text.strip()

            # Extract transfer fee/market value
            fee = columns[6].text.strip()

            # Append the data to the lists
            players.append(player_name)
            fees.append(fee)
        else:
            print("Skipping row: Not enough columns")

    # Print the scraped data
    for player, fee in zip(players, fees):
        print(f"Player: {player}, Fee/Market Value: {fee}")
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

Number of columns: 18
Number of columns: 2
Skipping row: Not enough columns
Number of columns: 1
Skipping row: Not enough columns
Number of columns: 2
Skipping row: Not enough columns
Number of columns: 1
Skipping row: Not enough columns
Number of columns: 2
Skipping row: Not enough columns
Number of columns: 1
Skipping row: Not enough columns
Number of columns: 18
Number of columns: 2
Skipping row: Not enough columns
Number of columns: 1
Skipping row: Not enough columns
Number of columns: 2
Skipping row: Not enough columns
Number of columns: 1
Skipping row: Not enough columns
Number of columns: 2
Skipping row: Not enough columns
Number of columns: 1
Skipping row: Not enough columns
Number of columns: 18
Number of columns: 2
Skipping row: Not enough columns
Number of columns: 1
Skipping row: Not enough columns
Number of columns: 2
Skipping row: Not enough columns
Number of columns: 1
Skipping row: Not enough columns
Number of columns: 2
Skipping row: Not enough columns
Number of column

In [20]:
import requests
from bs4 import BeautifulSoup
import csv

# URL of the page to scrape
url = "https://www.transfermarkt.com/transfers/transferrekorde/statistik?saison_id=alle&land_id=0&ausrichtung=&spielerposition_id=&altersklasse=&leihe=&w_s=&plus=1"

# Headers to mimic a browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Send a GET request to the URL
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the table containing the player data
    table = soup.find('table', {'class': 'items'})

    # Initialize a list to store the data
    data = []

    # Loop through each row in the table
    for row in table.find_all('tr')[1:]:  # Skip the header row
        columns = row.find_all('td')

        # Ensure the row has enough columns
        if len(columns) >= 7:  # Check if there are at least 7 columns
            # Extract player name
            player_name = columns[1].find('a').text.strip()

            # Extract transfer fee/market value
            fee = columns[6].text.strip()

            # Append the data as a dictionary to the list
            data.append({
                'Player Name': player_name,
                'Fee/Market Value': fee
            })

    # Save the data to a CSV file
    with open('transfer_data.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=['Player Name', 'Fee/Market Value'])

        # Write the header row
        writer.writeheader()

        # Write the data rows
        writer.writerows(data)

    print("Data saved to transfer_data.csv")
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

Data saved to transfer_data.csv


In [22]:
import requests
from bs4 import BeautifulSoup
import csv
import os

# URL of the page to scrape
base_url = "https://www.transfermarkt.com/transfers/transferrekorde/statistik"

# Headers to mimic a browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Initialize a list to store the data
data = []

# Loop through all pages
page = 1
while True:
    # Construct the URL for the current page
    url = f"{base_url}?saison_id=alle&land_id=0&ausrichtung=&spielerposition_id=&altersklasse=&leihe=&w_s=&plus=1&page={page}"

    # Send a GET request to the URL
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code != 200:
        print(f"Failed to retrieve page {page}. Status code: {response.status_code}")
        break

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the table containing the player data
    table = soup.find('table', {'class': 'items'})

    # Check if the table exists
    if not table:
        print(f"No table found on page {page}. Exiting.")
        break

    # Loop through each row in the table
    for row in table.find_all('tr')[1:]:  # Skip the header row
        columns = row.find_all('td')

        # Ensure the row has enough columns
        if len(columns) >= 7:  # Check if there are at least 7 columns
            # Extract player name
            player_name = columns[1].find('a').text.strip()

            # Extract transfer fee/market value
            fee = columns[6].text.strip()

            # Append the data as a dictionary to the list
            data.append({
                'Player Name': player_name,
                'Fee/Market Value': fee
            })

    # Check if there are more pages
    next_page = soup.find('li', {'class': 'tm-pagination__list-item--icon-next-page'})
    if not next_page:
        print("No more pages found. Exiting.")
        break

    # Move to the next page
    page += 1
    print(f"Scraped page {page}")

# Save the data to a CSV file in a different location (e.g., Desktop)
desktop_path = os.path.join(os.path.expanduser('~'), 'Desktop')
output_file = os.path.join(desktop_path, 'transfer_data.csv')

with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['Player Name', 'Fee/Market Value'])

    # Write the header row
    writer.writeheader()

    # Write the data rows
    writer.writerows(data)

print(f"Data saved to {output_file}. Total rows: {len(data)}")

Scraped page 2
Scraped page 3
Scraped page 4
Scraped page 5
Scraped page 6
Scraped page 7
Scraped page 8
Scraped page 9
Scraped page 10
No more pages found. Exiting.
Data saved to C:\Users\saikumar\Desktop\transfer_data.csv. Total rows: 250


In [23]:
import requests
from bs4 import BeautifulSoup
import csv
import os

# Base URL of the page to scrape
base_url = "https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop"

# Headers to mimic a browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Initialize a list to store the data
data = []

# Loop through all 20 pages
for page in range(1, 21):  # Pages 1 to 20
    # Construct the URL for the current page
    url = f"{base_url}?land_id=0&ausrichtung=alle&spielerposition_id=alle&altersklasse=alle&jahrgang=0&kontinent_id=0&plus=1&page={page}"

    # Send a GET request to the URL
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code != 200:
        print(f"Failed to retrieve page {page}. Status code: {response.status_code}")
        break

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the table containing the player data
    table = soup.find('table', {'class': 'items'})

    # Check if the table exists
    if not table:
        print(f"No table found on page {page}. Exiting.")
        break

    # Loop through each row in the table
    for row in table.find_all('tr')[1:]:  # Skip the header row
        columns = row.find_all('td')

        # Ensure the row has enough columns
        if len(columns) >= 8:  # Check if there are at least 8 columns
            # Extract player name
            player_name = columns[3].find('a').text.strip()

            # Extract market value
            market_value = columns[7].text.strip()

            # Append the data as a dictionary to the list
            data.append({
                'Player Name': player_name,
                'Market Value': market_value
            })

    print(f"Scraped page {page}")

# Save the data to a CSV file
desktop_path = os.path.join(os.path.expanduser('~'), 'Desktop')
output_file = os.path.join(desktop_path, 'transfermarkt_players.csv')

with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['Player Name', 'Market Value'])

    # Write the header row
    writer.writeheader()

    # Write the data rows
    writer.writerows(data)

print(f"Data saved to {output_file}. Total rows: {len(data)}")

Scraped page 1
Scraped page 2
Scraped page 3
Scraped page 4
Scraped page 5
Scraped page 6
Scraped page 7
Scraped page 8
Scraped page 9
Scraped page 10
Scraped page 11
Scraped page 12
Scraped page 13
Scraped page 14
Scraped page 15
Scraped page 16
Scraped page 17
Scraped page 18
Scraped page 19
Scraped page 20
Data saved to C:\Users\saikumar\Desktop\transfermarkt_players.csv. Total rows: 500


In [25]:
import requests
from bs4 import BeautifulSoup
import csv
import os

# Base URL of the page to scrape
base_url = "https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop"

# Headers to mimic a browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Initialize a list to store the data
data = []

# Loop through all 20 pages
for page in range(1, 21):  # Pages 1 to 20
    # Construct the URL for the current page
    url = f"{base_url}?land_id=0&ausrichtung=alle&spielerposition_id=alle&altersklasse=alle&jahrgang=0&kontinent_id=0&plus=1&page={page}"

    # Send a GET request to the URL
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code != 200:
        print(f"Failed to retrieve page {page}. Status code: {response.status_code}")
        break

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the table containing the player data
    table = soup.find('table', {'class': 'items'})

    # Check if the table exists
    if not table:
        print(f"No table found on page {page}. Exiting.")
        break

    # Loop through each row in the table
    for row in table.find_all('tr')[1:]:  # Skip the header row
        columns = row.find_all('td')

        # Ensure the row has enough columns
        if len(columns) >= 8:  # Check if there are at least 8 columns
            # Extract player name (4th column, index 3)
            player_name = columns[3].find('a').text.strip()

            # Extract market value (8th column, index 7)
            market_value = columns[7].text.strip()

            # Append the data as a dictionary to the list
            data.append({
                'Player Name': player_name,
                'Market Value': market_value
            })

    print(f"Scraped page {page}")

# Save the data to a CSV file
desktop_path = os.path.join(os.path.expanduser('~'), 'Desktop')
output_file = os.path.join(desktop_path, 'new_transfermarkt_players.csv')

with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['Player Name', 'Market Value'])

    # Write the header row
    writer.writeheader()

    # Write the data rows
    writer.writerows(data)

print(f"Data saved to {output_file}. Total rows: {len(data)}")

Scraped page 1
Scraped page 2
Scraped page 3
Scraped page 4
Scraped page 5
Scraped page 6
Scraped page 7
Scraped page 8
Scraped page 9
Scraped page 10
Scraped page 11
Scraped page 12
Scraped page 13
Scraped page 14
Scraped page 15
Scraped page 16
Scraped page 17
Scraped page 18
Scraped page 19
Scraped page 20
Data saved to C:\Users\saikumar\Desktop\new_transfermarkt_players.csv. Total rows: 500


In [30]:
import requests
from bs4 import BeautifulSoup
import csv
import os

# Base URL of the page to scrape
base_url = "https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop"

# Headers to mimic a browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Initialize a list to store the data
data = []

# Loop through all 20 pages
for page in range(1, 21):  # Pages 1 to 20
    # Construct the URL for the current page
    url = f"{base_url}?land_id=0&ausrichtung=alle&spielerposition_id=alle&altersklasse=alle&jahrgang=0&kontinent_id=0&plus=1&page={page}"

    # Send a GET request to the URL
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code != 200:
        print(f"Failed to retrieve page {page}. Status code: {response.status_code}")
        break

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the table containing the player data
    table = soup.find('table', {'class': 'items'})

    # Check if the table exists
    if not table:
        print(f"No table found on page {page}. Exiting.")
        break

    # Loop through each row in the table
    for row in table.find_all('tr')[1:]:  # Skip the header row
        columns = row.find_all('td')

        # Ensure the row has enough columns
        if len(columns) >= 14:  # Check if there are at least 7 columns
            # Extract player name (2nd column, index 1)
            player_name = columns[3].find('a').text.strip()

            # Extract market value (6th column, index 5)
            market_value = columns[8].text.strip()

            # Append the data as a dictionary to the list
            data.append({
                'Player Name': player_name,
                'Market Value': market_value
            })

    print(f"Scraped page {page}")

# Save the data to a CSV file
desktop_path = os.path.join(os.path.expanduser('~'), 'Desktop')
output_file = os.path.join(desktop_path, 'newlatest_transfermarkt_players.csv')

with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['Player Name', 'Market Value'])

    # Write the header row
    writer.writeheader()

    # Write the data rows
    writer.writerows(data)

print(f"Data saved to {output_file}. Total rows: {len(data)}")

Scraped page 1
Scraped page 2
Scraped page 3
Scraped page 4
Scraped page 5
Scraped page 6
Scraped page 7
Scraped page 8
Scraped page 9
Scraped page 10
Scraped page 11
Scraped page 12
Scraped page 13
Scraped page 14
Scraped page 15
Scraped page 16
Scraped page 17
Scraped page 18
Scraped page 19
Scraped page 20
Data saved to C:\Users\saikumar\Desktop\newlatest_transfermarkt_players.csv. Total rows: 500


In [33]:
import requests
from bs4 import BeautifulSoup
import csv
import os

# Base URL of the page to scrape
base_url = "https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop"

# Headers to mimic a browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Initialize a list to store the data
data = []

# Loop through all 20 pages
for page in range(1, 21):  # Pages 1 to 20
    # Construct the URL for the current page
    url = f"{base_url}?land_id=0&ausrichtung=alle&spielerposition_id=alle&altersklasse=u21&jahrgang=0&kontinent_id=0&plus=1&page={page}"

    # Send a GET request to the URL
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code != 200:
        print(f"Failed to retrieve page {page}. Status code: {response.status_code}")
        break

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the table containing the player data
    table = soup.find('table', {'class': 'items'})

    # Check if the table exists
    if not table:
        print(f"No table found on page {page}. Exiting.")
        break

    # Loop through each row in the table
    for row in table.find_all('tr')[1:]:  # Skip the header row
        columns = row.find_all('td')

        # Ensure the row has enough columns
        if len(columns) >= 14:  # Check if there are at least 8 columns
            # Extract player name (4th column, index 3)
            player_name = columns[3].find('a').text.strip()

            # Extract market value (8th column, index 7)
            market_value = columns[8].text.strip()

            # Append the data as a dictionary to the list
            data.append({
                'Player Name': player_name,
                'Market Value': market_value
            })

    print(f"Scraped page {page}")

# Save the data to a CSV file
desktop_path = os.path.join(os.path.expanduser('~'), 'Desktop')
output_file = os.path.join(desktop_path, 'newtransfermarkt_u21_players.csv')

with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['Player Name', 'Market Value'])

    # Write the header row
    writer.writeheader()

    # Write the data rows
    writer.writerows(data)

print(f"Data saved to {output_file}. Total rows: {len(data)}")

Scraped page 1
Scraped page 2
Scraped page 3
Scraped page 4
Scraped page 5
Scraped page 6
Scraped page 7
Scraped page 8
Scraped page 9
Scraped page 10
Scraped page 11
Scraped page 12
Scraped page 13
Scraped page 14
Scraped page 15
Scraped page 16
Scraped page 17
Scraped page 18
Scraped page 19
Scraped page 20
Data saved to C:\Users\saikumar\Desktop\newtransfermarkt_u21_players.csv. Total rows: 500


In [34]:
import requests
from bs4 import BeautifulSoup
import csv
import os

# Base URL of the page to scrape
base_url = "https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop"

# Headers to mimic a browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Initialize a list to store the data
data = []

# Loop through all 20 pages
for page in range(1, 21):  # Pages 1 to 20
    # Construct the URL for the current page
    url = f"{base_url}?land_id=0&ausrichtung=Torwart&spielerposition_id=alle&altersklasse=alle&jahrgang=0&kontinent_id=0&plus=1&page={page}"

    # Send a GET request to the URL
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code != 200:
        print(f"Failed to retrieve page {page}. Status code: {response.status_code}")
        break

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the table containing the player data
    table = soup.find('table', {'class': 'items'})

    # Check if the table exists
    if not table:
        print(f"No table found on page {page}. Exiting.")
        break

    # Loop through each row in the table
    for row in table.find_all('tr')[1:]:  # Skip the header row
        columns = row.find_all('td')

        # Ensure the row has enough columns
        if len(columns) >= 14:  # Check if there are at least 8 columns
            # Extract player name (4th column, index 3)
            player_name = columns[3].find('a').text.strip()

            # Extract market value (8th column, index 7)
            market_value = columns[8].text.strip()

            # Append the data as a dictionary to the list
            data.append({
                'Player Name': player_name,
                'Market Value': market_value
            })

    print(f"Scraped page {page}")

# Save the data to a CSV file
desktop_path = os.path.join(os.path.expanduser('~'), 'Desktop')
output_file = os.path.join(desktop_path, 'transfermarkt_goalkeepers.csv')

with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['Player Name', 'Market Value'])

    # Write the header row
    writer.writeheader()

    # Write the data rows
    writer.writerows(data)

print(f"Data saved to {output_file}. Total rows: {len(data)}")

Scraped page 1
Scraped page 2
Scraped page 3
Scraped page 4
Scraped page 5
Scraped page 6
Scraped page 7
Scraped page 8
Scraped page 9
Scraped page 10
Scraped page 11
Scraped page 12
Scraped page 13
Scraped page 14
Scraped page 15
Scraped page 16
Scraped page 17
Scraped page 18
Scraped page 19
Scraped page 20
Data saved to C:\Users\saikumar\Desktop\transfermarkt_goalkeepers.csv. Total rows: 500


In [35]:
import requests
from bs4 import BeautifulSoup
import csv
import os

# Base URL for defenders
base_url = "https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop"

# Headers to mimic a browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Initialize a list to store the data
data = []

# Loop through all 20 pages
for page in range(1, 21):  # Pages 1 to 20
    # Construct the URL for the current page
    url = f"{base_url}?land_id=0&ausrichtung=Abwehr&spielerposition_id=alle&altersklasse=alle&jahrgang=0&kontinent_id=0&plus=1&page={page}"

    # Send a GET request to the URL
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code != 200:
        print(f"Failed to retrieve page {page}. Status code: {response.status_code}")
        break

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the table containing the player data
    table = soup.find('table', {'class': 'items'})

    # Check if the table exists
    if not table:
        print(f"No table found on page {page}. Exiting.")
        break

    # Loop through each row in the table
    for row in table.find_all('tr')[1:]:  # Skip the header row
        columns = row.find_all('td')

        # Ensure the row has enough columns
        if len(columns) >= 14:  # Check if there are enough columns
            # Extract player name (4th column, index 3)
            player_name_tag = columns[3].find('a')
            player_name = player_name_tag.text.strip() if player_name_tag else "Unknown"

            # Extract market value (8th column, index 7)
            market_value = columns[8].text.strip()

            # Append the data as a dictionary to the list
            data.append({
                'Player Name': player_name,
                'Market Value': market_value
            })

    print(f"Scraped page {page}")

# Save the data to a CSV file
desktop_path = os.path.join(os.path.expanduser('~'), 'Desktop')
output_file = os.path.join(desktop_path, 'transfermarkt_defenders.csv')

with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['Player Name', 'Market Value'])

    # Write the header row
    writer.writeheader()

    # Write the data rows
    writer.writerows(data)

print(f"Data saved to {output_file}. Total rows: {len(data)}")

Scraped page 1
Scraped page 2
Scraped page 3
Scraped page 4
Scraped page 5
Scraped page 6
Scraped page 7
Scraped page 8
Scraped page 9
Scraped page 10
Scraped page 11
Scraped page 12
Scraped page 13
Scraped page 14
Scraped page 15
Scraped page 16
Scraped page 17
Scraped page 18
Scraped page 19
Scraped page 20
Data saved to C:\Users\saikumar\Desktop\transfermarkt_defenders.csv. Total rows: 500


In [36]:
import requests
from bs4 import BeautifulSoup
import csv
import os

# Base URL for midfielders
base_url = "https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop"

# Headers to mimic a browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Initialize a list to store the data
data = []

# Loop through all 20 pages
for page in range(1, 21):  # Pages 1 to 20
    # Construct the URL for the current page
    url = f"{base_url}?land_id=0&ausrichtung=Mittelfeld&spielerposition_id=alle&altersklasse=alle&jahrgang=0&kontinent_id=0&plus=1&page={page}"

    # Send a GET request to the URL
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code != 200:
        print(f"Failed to retrieve page {page}. Status code: {response.status_code}")
        break

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the table containing the player data
    table = soup.find('table', {'class': 'items'})

    # Check if the table exists
    if not table:
        print(f"No table found on page {page}. Exiting.")
        break

    # Loop through each row in the table
    for row in table.find_all('tr')[1:]:  # Skip the header row
        columns = row.find_all('td')

        # Ensure the row has enough columns
        if len(columns) >= 14:  # Check if there are enough columns
            # Extract player name (4th column, index 3)
            player_name_tag = columns[3].find('a')
            player_name = player_name_tag.text.strip() if player_name_tag else "Unknown"

            # Extract market value (8th column, index 7)
            market_value = columns[8].text.strip()

            # Append the data as a dictionary to the list
            data.append({
                'Player Name': player_name,
                'Market Value': market_value
            })

    print(f"Scraped page {page}")

# Save the data to a CSV file
desktop_path = os.path.join(os.path.expanduser('~'), 'Desktop')
output_file = os.path.join(desktop_path, 'transfermarkt_midfielders.csv')

with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['Player Name', 'Market Value'])

    # Write the header row
    writer.writeheader()

    # Write the data rows
    writer.writerows(data)

print(f"Data saved to {output_file}. Total rows: {len(data)}")

Scraped page 1
Scraped page 2
Scraped page 3
Scraped page 4
Scraped page 5
Scraped page 6
Scraped page 7
Scraped page 8
Scraped page 9
Scraped page 10
Scraped page 11
Scraped page 12
Scraped page 13
Scraped page 14
Scraped page 15
Scraped page 16
Scraped page 17
Scraped page 18
Scraped page 19
Scraped page 20
Data saved to C:\Users\saikumar\Desktop\transfermarkt_midfielders.csv. Total rows: 500


In [37]:
import requests
from bs4 import BeautifulSoup
import csv
import os

# Base URL for forwards
base_url = "https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop"

# Headers to mimic a browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Initialize a list to store the data
data = []

# Loop through all 20 pages
for page in range(1, 21):  # Pages 1 to 20
    # Construct the URL for the current page
    url = f"{base_url}?land_id=0&ausrichtung=Sturm&spielerposition_id=alle&altersklasse=alle&jahrgang=0&kontinent_id=0&plus=1&page={page}"

    # Send a GET request to the URL
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code != 200:
        print(f"Failed to retrieve page {page}. Status code: {response.status_code}")
        break

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the table containing the player data
    table = soup.find('table', {'class': 'items'})

    # Check if the table exists
    if not table:
        print(f"No table found on page {page}. Exiting.")
        break

    # Loop through each row in the table
    for row in table.find_all('tr')[1:]:  # Skip the header row
        columns = row.find_all('td')

        # Ensure the row has enough columns
        if len(columns) >= 14:  # Check if there are enough columns
            # Extract player name (4th column, index 3)
            player_name_tag = columns[3].find('a')
            player_name = player_name_tag.text.strip() if player_name_tag else "Unknown"

            # Extract market value (8th column, index 7)
            market_value = columns[8].text.strip()

            # Append the data as a dictionary to the list
            data.append({
                'Player Name': player_name,
                'Market Value': market_value
            })

    print(f"Scraped page {page}")

# Save the data to a CSV file
desktop_path = os.path.join(os.path.expanduser('~'), 'Desktop')
output_file = os.path.join(desktop_path, 'transfermarkt_forwards.csv')

with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['Player Name', 'Market Value'])

    # Write the header row
    writer.writeheader()

    # Write the data rows
    writer.writerows(data)

print(f"Data saved to {output_file}. Total rows: {len(data)}")

Scraped page 1
Scraped page 2
Scraped page 3
Scraped page 4
Scraped page 5
Scraped page 6
Scraped page 7
Scraped page 8
Scraped page 9
Scraped page 10
Scraped page 11
Scraped page 12
Scraped page 13
Scraped page 14
Scraped page 15
Scraped page 16
Scraped page 17
Scraped page 18
Scraped page 19
Scraped page 20
Data saved to C:\Users\saikumar\Desktop\transfermarkt_forwards.csv. Total rows: 500


In [39]:
import pandas as pd
import glob

# Use raw string (r"") or double backslashes to avoid Unicode escape issues
csv_files = glob.glob(r"C:\Users\saikumar\Desktop\merge_all\*.csv")

# Create an empty list to store DataFrames
dataframes = []

# Loop through the files and read each one
for file in csv_files:
    df = pd.read_csv(file)  # Read each CSV file
    dataframes.append(df)

# Merge all DataFrames into one
merged_df = pd.concat(dataframes, ignore_index=True)

# Save to a new CSV file
output_file = r"C:\Users\saikumar\Desktop\merge_all\merged_players.csv"
merged_df.to_csv(output_file, index=False)

print(f"All files merged successfully into '{output_file}'")


All files merged successfully into 'C:\Users\saikumar\Desktop\merge_all\merged_players.csv'


In [46]:
import csv
import unicodedata

# Function to normalize text
def normalize_text(text):
    # Normalize the text (NFKD normalization separates characters and diacritics)
    normalized = unicodedata.normalize('NFKD', text)
    # Encode to ASCII, ignoring non-ASCII characters, then decode back to string
    ascii_text = normalized.encode('ascii', 'ignore').decode('ascii')
    return ascii_text

# File paths
input_file = r'C:\Users\saikumar\Desktop\merged_players.csv'  # Use raw string (r) for Windows paths
output_file = r'C:\Users\saikumar\Desktop\merged_players_normalized.csv'  # Output file

# Read the CSV file
with open(input_file, mode='r', encoding='utf-8') as infile, open(output_file, mode='w', encoding='utf-8', newline='') as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)

    # Write the header (if any)
    header = next(reader)
    writer.writerow(header)

    # Process each row
    for row in reader:
        normalized_row = [normalize_text(cell) for cell in row]
        writer.writerow(normalized_row)

print(f"Processed file saved as {output_file}")

Processed file saved as C:\Users\saikumar\Desktop\merged_players_normalized.csv


In [52]:
import pandas as pd
from fuzzywuzzy import process

# File paths
main_file = r'C:\Users\saikumar\Desktop\merged_players_normalized.csv'
additional_file = r'C:\Users\saikumar\Desktop\MSc Data Analytics\football_thesis\all_fb_players_fast.csv'
output_file = r'C:\Users\saikumar\Desktop\merged_players_with_market_values.csv'

# Load the CSV files into DataFrames
df_main = pd.read_csv(main_file)
df_additional = pd.read_csv(additional_file)

# Print column names for debugging
print("Main file columns:", df_main.columns.tolist())
print("Additional file columns:", df_additional.columns.tolist())

# Strip leading/trailing spaces from column names
df_main.columns = df_main.columns.str.strip()
df_additional.columns = df_additional.columns.str.strip()

# Check if the required columns exist
required_columns_main = ['Player Name']  # Update with the correct column name
required_columns_additional = ['Player Name', 'Market Value']  # Update with the correct column names

for col in required_columns_main:
    if col not in df_main.columns:
        raise KeyError(f"Column '{col}' not found in the main file.")

for col in required_columns_additional:
    if col not in df_additional.columns:
        raise KeyError(f"Column '{col}' not found in the additional file.")

# Function to find the best match for a name in the additional DataFrame
def find_best_match(name, choices, threshold=80):
    match, score = process.extractOne(name, choices)
    return match if score >= threshold else None

# Create a dictionary of player names and market values from the additional file
name_to_market_value = dict(zip(df_additional['Player Name'], df_additional['Market Value']))

# Add a new column to the main DataFrame for matched market values
df_main['Market Value'] = None

# Iterate through the main DataFrame and find matches
for index, row in df_main.iterrows():
    player_name = row['Player Name']
    best_match = find_best_match(player_name, name_to_market_value.keys())
    if best_match:
        df_main.at[index, 'Market Value'] = name_to_market_value[best_match]

# Save the updated DataFrame to a new CSV file
df_main.to_csv(output_file, index=False)

print(f"Updated file saved as {output_file}")

Main file columns: ['Player Name ', 'Market Value']
Additional file columns: ['Rk', 'Player Name', 'Nation', 'Position', 'Squad', 'Comp', 'Age', 'Birth Year', 'Games', 'Starts', 'Minutes', 'Nintys', 'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'Yellow', 'Red', 'xG', 'npxG', 'xAG', 'npxG+xAG', 'PrgC', 'PrgP', 'PrgR', 'Glss', 'Asts', 'G+As', 'G-PKs', 'G+A-PK', 'xGs', 'xAGs', 'xG+xAG', 'npxGs', 'npxG+xAG.1', 'Matches', 'Market Value']
Updated file saved as C:\Users\saikumar\Desktop\merged_players_with_market_values.csv


In [53]:
print("Main file head:")
print(df_main.head())

print("Additional file head:")
print(df_additional.head())

Main file head:
       Player Name Market Value
0     Lamine Yamal          NaN
1  Jude Bellingham          NaN
2    Florian Wirtz          NaN
3    Jamal Musiala          NaN
4             Gavi          NaN
Additional file head:
  Rk        Player Name   Nation Position        Squad                Comp  \
0  1         Max Aarons  eng ENG       DF  Bournemouth  eng Premier League   
1  2         Max Aarons  eng ENG       MF     Valencia          es La Liga   
2  3     Rodrigo Abajas   es ESP       DF     Valencia          es La Liga   
3  4     James Abankwah   ie IRL    DF,MF      Udinese          it Serie A   
4  5  Keyliane Abdallah   fr FRA       FW    Marseille          fr Ligue 1   

      Age Birth Year Games Starts  ... G+As G-PKs G+A-PK   xGs xAGs xG+xAG  \
0  25-051       2000     3      1  ...    0     0      0     0    0      0   
1  25-051       2000     1      0  ...    0     0      0     0    0      0   
2  21-288       2003     1      1  ...    0     0      0   0.1    0

In [54]:
import pandas as pd

# File paths
main_file = r'C:\Users\saikumar\Desktop\merged_players_normalized.csv'
additional_file = r'C:\Users\saikumar\Desktop\MSc Data Analytics\football_thesis\all_fb_players_fast.csv'
output_file = r'C:\Users\saikumar\Desktop\merged_players_with_new_market_values.csv'

# Load the CSV files into DataFrames
df_main = pd.read_csv(main_file)
df_additional = pd.read_csv(additional_file)

# Print column names for debugging
print("Main file columns:", df_main.columns.tolist())
print("Additional file columns:", df_additional.columns.tolist())

# Strip leading/trailing spaces from column names
df_main.columns = df_main.columns.str.strip()
df_additional.columns = df_additional.columns.str.strip()

# Check if the required columns exist
required_columns_main = ['Player Name']  # Update with the correct column name
required_columns_additional = ['Player Name', 'Market Value']  # Update with the correct column names

for col in required_columns_main:
    if col not in df_main.columns:
        raise KeyError(f"Column '{col}' not found in the main file.")

for col in required_columns_additional:
    if col not in df_additional.columns:
        raise KeyError(f"Column '{col}' not found in the additional file.")

# Create a dictionary of player names and market values from the additional file
name_to_market_value = dict(zip(df_additional['Player Name'], df_additional['Market Value']))

# Add a new column to the main DataFrame for matched market values
df_main['Market Value'] = None

# Iterate through the main DataFrame and assign market values for exact matches
for index, row in df_main.iterrows():
    player_name = row['Player Name']
    if player_name in name_to_market_value:
        df_main.at[index, 'Market Value'] = name_to_market_value[player_name]

# Save the updated DataFrame to a new CSV file
df_main.to_csv(output_file, index=False)

print(f"Updated file saved as {output_file}")

Main file columns: ['Player Name ', 'Market Value']
Additional file columns: ['Rk', 'Player Name', 'Nation', 'Position', 'Squad', 'Comp', 'Age', 'Birth Year', 'Games', 'Starts', 'Minutes', 'Nintys', 'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'Yellow', 'Red', 'xG', 'npxG', 'xAG', 'npxG+xAG', 'PrgC', 'PrgP', 'PrgR', 'Glss', 'Asts', 'G+As', 'G-PKs', 'G+A-PK', 'xGs', 'xAGs', 'xG+xAG', 'npxGs', 'npxG+xAG.1', 'Matches', 'Market Value']
Updated file saved as C:\Users\saikumar\Desktop\merged_players_with_new_market_values.csv


In [56]:
common_names = set(df_main['Player Name']).intersection(set(df_additional['Player Name']))
print("Common player names:", len(common_names))

Common player names: 934


In [57]:
import re

def normalize_name(name):
    # Remove special characters, convert to lowercase, and strip spaces
    name = re.sub(r'[^a-zA-Z0-9\s]', '', name)  # Remove special characters
    name = name.lower().strip()  # Convert to lowercase and strip spaces
    return name

# Normalize player names in both DataFrames
df_main['Player Name'] = df_main['Player Name'].apply(normalize_name)
df_additional['Player Name'] = df_additional['Player Name'].apply(normalize_name)

In [58]:
common_names = set(df_main['Player Name']).intersection(set(df_additional['Player Name']))
print("Common player names:", len(common_names))

Common player names: 946


In [61]:
import pandas as pd
from fuzzywuzzy import process
import re

# File paths
main_file = r'C:\Users\saikumar\Desktop\merged_players_normalized.csv'
additional_file = r'C:\Users\saikumar\Desktop\MSc Data Analytics\football_thesis\all_fb_players_fast.csv'
output_file = r'C:\Users\saikumar\Desktop\final_merged_players_with_market_values.csv'

# Load the CSV files into DataFrames
df_main = pd.read_csv(main_file)
df_additional = pd.read_csv(additional_file)

# Print column names for debugging
print("Main file columns:", df_main.columns.tolist())
print("Additional file columns:", df_additional.columns.tolist())

# Strip leading/trailing spaces from column names
df_main.columns = df_main.columns.str.strip()
df_additional.columns = df_additional.columns.str.strip()

# Check if the required columns exist
required_columns_main = ['Player Name']  # Update with the correct column name
required_columns_additional = ['Player Name', 'Market Value']  # Update with the correct column names

for col in required_columns_main:
    if col not in df_main.columns:
        raise KeyError(f"Column '{col}' not found in the main file.")

for col in required_columns_additional:
    if col not in df_additional.columns:
        raise KeyError(f"Column '{col}' not found in the additional file.")

# Normalize player names
def normalize_name(name):
    # Remove special characters, convert to lowercase, and strip spaces
    name = re.sub(r'[^a-zA-Z0-9\s]', '', name)  # Remove special characters
    name = name.lower().strip()  # Convert to lowercase and strip spaces
    return name

df_main['Player Name'] = df_main['Player Name'].apply(normalize_name)
df_additional['Player Name'] = df_additional['Player Name'].apply(normalize_name)

# Create a dictionary of player names and market values from the additional file
name_to_market_value = dict(zip(df_additional['Player Name'], df_additional['Market Value']))

# Add a new column to the main DataFrame for matched market values
df_main['Market Value'] = None

# Counters for exact and fuzzy matches
exact_matches = 0
fuzzy_matches = 0

# Step 1: Exact matching
print("Starting exact matching...")
for index, row in df_main.iterrows():
    player_name = row['Player Name']
    if player_name in name_to_market_value:
        df_main.at[index, 'Market Value'] = name_to_market_value[player_name]
        exact_matches += 1

print(f"Exact matches found: {exact_matches}")

# Step 2: Fuzzy matching for remaining players
print("Starting fuzzy matching...")
total_players = len(df_main)
for index, row in df_main.iterrows():
    if pd.isna(row['Market Value']):  # Only process rows without a market value
        player_name = row['Player Name']
        # Find the best match in the additional file
        best_match, score = process.extractOne(player_name, name_to_market_value.keys())
        if score >= 80:  # Adjust threshold as needed
            df_main.at[index, 'Market Value'] = name_to_market_value[best_match]
            fuzzy_matches += 1
            print(f"Fuzzy Match: {player_name} -> {best_match} (Score: {score})")

    # Print progress
    if (index + 1) % 100 == 0:  # Print progress every 100 players
        print(f"Processed {index + 1}/{total_players} players...")

print(f"Fuzzy matches found: {fuzzy_matches}")

# Save the updated DataFrame to a new CSV file
df_main.to_csv(output_file, index=False)

print(f"Updated file saved as {output_file}")

Main file columns: ['Player Name ', 'Market Value']
Additional file columns: ['Rk', 'Player Name', 'Nation', 'Position', 'Squad', 'Comp', 'Age', 'Birth Year', 'Games', 'Starts', 'Minutes', 'Nintys', 'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'Yellow', 'Red', 'xG', 'npxG', 'xAG', 'npxG+xAG', 'PrgC', 'PrgP', 'PrgR', 'Glss', 'Asts', 'G+As', 'G-PKs', 'G+A-PK', 'xGs', 'xAGs', 'xG+xAG', 'npxGs', 'npxG+xAG.1', 'Matches', 'Market Value']
Starting exact matching...
Exact matches found: 1194
Starting fuzzy matching...
Fuzzy Match: lamine yamal -> lamine yamal (Score: 100)
Fuzzy Match: jude bellingham -> jude bellingham (Score: 100)
Fuzzy Match: florian wirtz -> florian wirtz (Score: 100)
Fuzzy Match: jamal musiala -> jamal musiala (Score: 100)
Fuzzy Match: gavi -> gavi (Score: 100)
Fuzzy Match: xavi simons -> xavi simons (Score: 100)
Fuzzy Match: pau cubarsi -> pau cubars (Score: 95)
Fuzzy Match: warren zaireemery -> warren zareemery (Score: 97)
Fuzzy Match: joao neves -> joo neves (Score: 95)


In [64]:
import pandas as pd
from rapidfuzz import process
import re

# File paths
main_file = r'C:\Users\saikumar\Desktop\MSc Data Analytics\football_thesis\all_fb_players_fast.csv'
additional_file = r'C:\Users\saikumar\Desktop\merged_players_normalized.csv'
output_file = r'C:\Users\saikumar\Desktop\merged_players_lasttry_with_market_values.csv'

# Load the CSV files into DataFrames
df_main = pd.read_csv(main_file)
df_additional = pd.read_csv(additional_file)

# Print column names for debugging
print("Main file columns:", df_main.columns.tolist())
print("Additional file columns:", df_additional.columns.tolist())

# Strip leading/trailing spaces from column names
df_main.columns = df_main.columns.str.strip()
df_additional.columns = df_additional.columns.str.strip()

# Check if the required columns exist
required_columns_main = ['Player Name']  # Update with the correct column name
required_columns_additional = ['Player Name', 'Market Value']  # Update with the correct column names

for col in required_columns_main:
    if col not in df_main.columns:
        raise KeyError(f"Column '{col}' not found in the main file.")

for col in required_columns_additional:
    if col not in df_additional.columns:
        raise KeyError(f"Column '{col}' not found in the additional file.")

# Normalize player names
def normalize_name(name):
    # Remove special characters, convert to lowercase, and strip spaces
    name = re.sub(r'[^a-zA-Z0-9\s]', '', name)  # Remove special characters
    name = name.lower().strip()  # Convert to lowercase and strip spaces
    return name

df_main['Player Name'] = df_main['Player Name'].apply(normalize_name)
df_additional['Player Name'] = df_additional['Player Name'].apply(normalize_name)

# Create a dictionary of player names and market values from the additional file
name_to_market_value = dict(zip(df_additional['Player Name'], df_additional['Market Value']))

# Add a new column to the main DataFrame for matched market values
df_main['Market Value'] = None

# Counters for exact and fuzzy matches
exact_matches = 0
fuzzy_matches = 0

# Step 1: Exact matching
print("Starting exact matching...")
for index, row in df_main.iterrows():
    player_name = row['Player Name']
    if player_name in name_to_market_value:
        df_main.at[index, 'Market Value'] = name_to_market_value[player_name]
        exact_matches += 1
        print(f"Exact Match: {player_name} -> {player_name}")

print(f"Exact matches found: {exact_matches}")

# Step 2: Fuzzy matching for remaining players
print("Starting fuzzy matching...")
total_players = len(df_main)
for index, row in df_main.iterrows():
    if pd.isna(row['Market Value']):  # Only process rows without a market value
        player_name = row['Player Name']
        # Find the best match in the additional file
        best_match, score, _ = process.extractOne(player_name, name_to_market_value.keys())
        if score >= 80:  # Adjust threshold as needed
            df_main.at[index, 'Market Value'] = name_to_market_value[best_match]
            fuzzy_matches += 1
            print(f"Fuzzy Match: {player_name} -> {best_match} (Score: {score})")

    # Print progress
    if (index + 1) % 100 == 0:  # Print progress every 100 players
        print(f"Processed {index + 1}/{total_players} players...")

print(f"Fuzzy matches found: {fuzzy_matches}")

# Step 3: Merge all columns from both files
# Rename columns in the additional file to avoid conflicts (e.g., 'Market Value' -> 'Market Value_Additional')
df_additional = df_additional.rename(columns={'Market Value': 'Market Value_Additional'})

# Merge the main DataFrame with the additional DataFrame on 'Player Name'
merged_df = pd.merge(df_main, df_additional, on='Player Name', how='left')

# Save the updated DataFrame to a new CSV file
merged_df.to_csv(output_file, index=False)

print(f"Updated file saved as {output_file}")

Main file columns: ['Rk', 'Player Name', 'Nation', 'Position', 'Squad', 'Comp', 'Age', 'Birth Year', 'Games', 'Starts', 'Minutes', 'Nintys', 'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'Yellow', 'Red', 'xG', 'npxG', 'xAG', 'npxG+xAG', 'PrgC', 'PrgP', 'PrgR', 'Glss', 'Asts', 'G+As', 'G-PKs', 'G+A-PK', 'xGs', 'xAGs', 'xG+xAG', 'npxGs', 'npxG+xAG.1', 'Matches', 'Market Value']
Additional file columns: ['Player Name ', 'Market Value']
Starting exact matching...
Exact Match: max aarons -> max aarons
Exact Match: max aarons -> max aarons
Exact Match: matthis abline -> matthis abline
Exact Match: abner -> abner
Exact Match: zakaria aboukhlal -> zakaria aboukhlal
Exact Match: abdel abqar -> abdel abqar
Exact Match: tammy abraham -> tammy abraham
Exact Match: tammy abraham -> tammy abraham
Exact Match: akor adams -> akor adams
Exact Match: akor adams -> akor adams
Exact Match: che adams -> che adams
Exact Match: tyler adams -> tyler adams
Exact Match: tosin adarabioyo -> tosin adarabioyo
Exact 